# 1. Numba

Numba překládá vybrané části Python kódu za běhu (JIT) přes LLVM. Největší přínos má u numerických funkcí s cykly a NumPy poli.

V tomto notebooku si ukážeme:
- použití `@numba.jit`,
- stencil výpočty přes `numba.stencil`.

In [ ]:
#!pip install numba

## 1.1 Dekorátor `jit`

`numba.jit` lze použít jako dekorátor i jako funkci. V praxi se nejčastěji pracuje s parametry:
- `nopython=True` pro čistě zkompilovanou cestu bez přepnutí do objektového módu,
- `signature` pro explicitní typy vstupu a výstupu,
- `parallel=True` pro pokus o paralelizaci smyček,
- `fastmath=True` pro rychlejší (ale méně striktní) matematické optimalizace,
- `cache=True` pro uložení překladu mezi běhy.

In [ ]:
def my_dot_python(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

In [ ]:
import numpy as np
a = np.random.rand(1000000)
b = np.random.rand(1000000)

In [ ]:
%time c = my_dot_python(a, b)

In [ ]:
import numba
my_dot_numba = numba.jit(my_dot_python)

In [ ]:
%time c = my_dot_numba(a, b)

In [ ]:
@numba.jit(signature_or_function='float64(float64[:], float64[:])',
           nopython=True,
           fastmath=True,
           locals={'result': numba.float64})
def my_dot_numba2(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

In [ ]:
%time c = my_dot_numba2(a, b)

## 1.2 `Numba.stencil`

`stencil` je pohodlný zápis lokálních výpočtů nad okolím prvku (typicky u obrazů, mřížek nebo PDE aproximací).

In [ ]:
from numba import stencil

@stencil()
def kernel1(a):
    return 0.25 * (a[0, 1] + a[1, 0] + a[0, -1] + a[-1, 0])

In [ ]:
import numpy as np
n = 5
input_arr = np.arange(n*n).reshape((n, n))
# pad with zeros
input_arr = np.pad(input_arr, 1, mode='constant', constant_values=0)
print(input_arr)

In [ ]:
kernel1(input_arr)

Základní stencil funkci můžeme dále zkompilovat přes `jit`, aby běžela rychleji i pro větší vstupy.

In [ ]:
from numba import jit

@jit
def kernel2(input_arr):
    @stencil
    def kernel1(a):
        return 0.25 * (a[0, 1] + a[1, 0] + a[0, -1] + a[-1, 0])
    return kernel1(input_arr)

In [ ]:
kernel2(input_arr)